# Lab 3 - The Matrix Form and The Duality Theory

<b>Information on group members:</b><br>
1) 144091, Bartosz Maślanka <br>
2) Student ID, Kajetan Kubik

In [1]:
from pulp import *  
import numpy as np
import pandas as pd
import itertools
from IPython.display import display
import numpy.ma as ma


# 1) The Matrix Form - Fundamental Insight (finish this part to get 3.0)

1.1) Given is the below (primal) linear problem:

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

1.2) Use the PuLP library to solve the above problem. Identify the optimal solution: the values for basic variables and the corresponding value for the objective function.

In [2]:
model = LpProblem(name="part_1_problem", sense=LpMaximize)
var_names = ["1", "2", "3"]
x = LpVariable.dicts("x", var_names, 0)
const_names = ["LE1", "LE2", "LE3", "LE4"]
sense = [-1, -1, -1, -1]
coefs = [[1,1,0],
         [0,2,1],
         [3,0,2],
         [1,1,1]]
rhs = [10, 12, 15, 20] 
for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(3)])
    model += LpConstraint(e=expr, sense = s, name = cn, rhs = r)
obj_coefs = [4,2,3]   
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(3)])
model

part_1_problem:
MAXIMIZE
4*x_1 + 2*x_2 + 3*x_3 + 0
SUBJECT TO
LE1: x_1 + x_2 <= 10

LE2: 2 x_2 + x_3 <= 12

LE3: 3 x_1 + 2 x_3 <= 15

LE4: x_1 + x_2 + x_3 <= 20

VARIABLES
x_1 Continuous
x_2 Continuous
x_3 Continuous

In [3]:
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for name, constraint in model.constraints.items():  
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 31.428571379999998
LE1: 0.0
LE2: 5.999999985739635e-08
LE3: -7.999999906971311e-08
LE4: -9.14285714


In [4]:
for n in var_names: print(n,'=',x[n].value())


1 = 4.4285714
2 = 5.5714286
3 = 0.85714286


1.3) In this exercise, you are asked to identify ALL basic (feasible and not) solutions to the above problem. We will do this naively. Specifically, you are asked to use the fundamental insight to build a final simplex tableau for each possible base. Therefore, you need first to initialize the data: c, b, A matrixes, and it is suggested to initialize the auxiliary matrix M defined as M = A + (concatenate) I (identity matrix). Note that the problem should be formulated in the augmented form. Then, you have to iterate over each possible base B, compute B-1, and other relevant parts for the simplex tableau.<br><br>
a) Identify the optimal solution using the optimality condition; print it (Z value and values for basic variables); compare thus derived solution with the optimum found using the PuLP library (obviously, both solutions should be the same). <br>
b) Count the number of feasible and infeasible solutions. How many (all) basic solutions to the problem can be identified? <br><br>
It is suggested to use the NumPy library for performing matrix operations. 

In [5]:
c = np.array([4,2,3])
b = np.array([10,12,15,20])
A = np.array([[1,1,0],
             [0,2,1],
             [3,0,2],
             [1,1,1]])
M = np.concatenate((A,np.identity(4)), axis=1)


In [6]:
M

array([[1., 1., 0., 1., 0., 0., 0.],
       [0., 2., 1., 0., 1., 0., 0.],
       [3., 0., 2., 0., 0., 1., 0.],
       [1., 1., 1., 0., 0., 0., 1.]])

<b> Important note: the below is just a proposition. You can solve the problem in your own way. </b>

You can define an auxiliary method constructing a final simplex tableau for a given base.  Here, "base" is a list of columns (integers) for the base. Note that the functions in python can return multiple objects and you can use this functionality to return<br>
- the inversed base<br>
- coefficients in the row 0 for slack variables<br>
- right side values (except the objective function value)<br>
- the objective function value<br>
- the coefficients for decision variables in row 0 <br>
- the coefficients for decision variables in rows 1+<br>

Note that if BI cannot be built (it is possible), the method may return None in order to notify the executive method about this exception. 



In [10]:
def getFinalTableau(base, c, b, A, M):
    new_c = c * -1
    rs = np.concatenate(([0], b))
    tabs = []
    tableau = np.vstack((np.concatenate((new_c, [0, 0, 0, 0])), M))
    tableau = np.append(tableau, rs.reshape((-1,1)),axis=1)
    tabs.append(tableau.copy())
    check_base = set([1,2,3,4,5,6,7])
    check_base.difference_update(set(base))
    while check_base:
        pivot_col = np.argmin(ma.masked_where(tableau[0]==0, tableau[0]))
        mrt = []
        last_item = len(tableau[0]) - 1
        for i in range(len(tableau)):
            if i == pivot_col or tableau[i][last_item] / tableau[i][pivot_col] < 0:
                mrt.append(np.inf)
            else:
                mrt.append(tableau[i][last_item] / tableau[i][pivot_col])
        selected_variable = np.argmin(mrt)
        tableau[selected_variable] = tableau[selected_variable] / tableau[selected_variable][pivot_col]
        base[selected_variable-1] = pivot_col+1
        for i in range(len(tableau)):
            if i == selected_variable:
                continue
            pivot_num = tableau[i][pivot_col] * -1
            tableau[i] = (tableau[selected_variable] * pivot_num) + tableau[i]
        
        tabs.append(tableau.copy())
        check_base.difference_update(set(base))

    print(base)
    return tabs,base


In [11]:
base = [4,5,6,7]
results,base = getFinalTableau(base, c, b, A, M)

[3, 2, 1, 7]


<ipython-input-10-4d20d5ee04e1>:15: RuntimeWarning: divide by zero encountered in double_scalars
  if i == pivot_col or tab[i][last_item] / tab[i][pivot_col] < 0:
<ipython-input-10-4d20d5ee04e1>:18: RuntimeWarning: divide by zero encountered in double_scalars
  mrt.append(tab[i][last_item] / tab[i][pivot_col])


In [12]:
for i in results:
    print(np.round(i,decimals=2))
    print('\n')

[[-4. -2. -3.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  1.  0.  0.  0. 10.]
 [ 0.  2.  1.  0.  1.  0.  0. 12.]
 [ 3.  0.  2.  0.  0.  1.  0. 15.]
 [ 1.  1.  1.  0.  0.  0.  1. 20.]]


[[ 0.   -2.   -0.33  0.    0.    1.33  0.   20.  ]
 [ 0.    1.   -0.67  1.    0.   -0.33  0.    5.  ]
 [ 0.    2.    1.    0.    1.    0.    0.   12.  ]
 [ 1.    0.    0.67  0.    0.    0.33  0.    5.  ]
 [ 0.    1.    0.33  0.    0.   -0.33  1.   15.  ]]


[[ 0.    0.    0.67  0.    1.    1.33  0.   32.  ]
 [ 0.    0.   -1.17  1.   -0.5  -0.33  0.   -1.  ]
 [ 0.    1.    0.5   0.    0.5   0.    0.    6.  ]
 [ 1.    0.    0.67  0.    0.    0.33  0.    5.  ]
 [ 0.    0.   -0.17  0.   -0.5  -0.33  1.    9.  ]]


[[ 0.    0.    0.    0.57  0.71  1.14  0.   31.43]
 [-0.   -0.    1.   -0.86  0.43  0.29 -0.    0.86]
 [ 0.    1.    0.    0.43  0.29 -0.14  0.    5.57]
 [ 1.    0.    0.    0.57 -0.29  0.14  0.    4.43]
 [ 0.    0.    0.   -0.14 -0.43 -0.29  1.    9.14]]




In [13]:
for i in range(len(base)-1):
    print(base[i],'=',results[-1][:,-1][np.argmax(results[-1][:,base[i]-1])])

3 = 0.8571428571428572
2 = 5.571428571428571
1 = 4.428571428571429


## 2) The Duality Theory (finish this part + part 1 + to get 5.0)

2.1) Model the dual problem to the above solved primal one, using the PuLP library. Then, solve it and compare the derived optimum with the optimum for the primal problem. Are they equal?

In [ ]:
### Model here:
### TODO


In [ ]:
### Solve here:
### TODO


2.2) This exercise is based on the exercise 1.3 (copy & paste solution). Here, you are asked to iterate over all basic solutions (as in 1.3) and store them along with their complementary dual solutions. Solutions should be stored in the PRIMAL_DUAL_SOLUTIONS list and sorted according to the objective value Z = W. Analyze their optimality and feasibility. Finally, you have to display all basic solutions wlong with their complementary solutions (you can use the provided piece of code written using the pandas library). <br><br>

PRIMAL_DUAL_SOLUTIONS is defined as a table consisting of n rows, where n is the number of basic solutions to the problem, and 21 columns. The columns are defined as follows:<br>
Col. 1: The objective value Z<br>
Col. 2-4: The values for decision variables (primal solution)<br>
Col. 5-8: The values for slack variables (primal solution)<br>
Col. 9: P_F = Y or N, Y/N = primal solution is feasible/infeasible<br>
Col. 10: P_O = Y or N, Y/N = primal solution is optimal/is not optimal<br>
Col. 11: P_STATE = -/suboptimal/superoptimal/optimal; depends on P_F and P_O (primal)<br>
Col. 12: D_STATE = -/suboptimal/superoptimal/optimal; depends on D_F and D_O (dual)<br>
Col. 13: D_F = Y or N, Y/N = dual solution is feasible/infeasible<br>
Col. 14: D_O = Y or N, Y/N = dual solution is optimal/is not optimal<br>
Col. 15-18: The values for decision variables (dual solution)<br>
Col. 19-21: The values for surplus variables (dual solution)<br><br>

Reminder: sort solutions according to Z; analyze how their states change with the increase of Z.

In [ ]:
### TODO 

### DISPLAY STORED PAIRS OF SOLUTIONS
#df = pd.DataFrame(PRIMAL_DUAL_SOLUTIONS, columns = ["Z", "x1", "x2", "x3", "slack1", "slack2", "slack3", "slack4", "P_F", "P_O", "P_STATE", "D_STATE", "D_F", "F_O",
#                                                   "y1", "y2", "y3", "y4", "sur1", "sur2", "sur3"])

#display(df.style.set_properties(**{
#        'width': '230px',
#        'max-width': '230px',
#    }))